<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/static/images/logo-v4.svg" width=190/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/project_configuration/queue_management.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/project_configuration/queue_management.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Queue Management

* The queue is used to task labelers with specific assets
* We can do any of the following:
    * Set the number of times we want an image labeled
        * Referred to as `Consensus` in the front end
    * Set the order of items in the queue
    * Set the percent of assets to review

In [ ]:
!pip install labelbox
!pip install numpy

In [2]:
from labelbox import Client, LabelingFrontend, OntologyBuilder, MediaType
from labelbox.schema.queue_mode import QueueMode
import numpy as np
from io import BytesIO
import os
import uuid

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [3]:
# Add your api key
API_KEY = None
client = Client(api_key=API_KEY)

### Set up demo project

In [18]:
## Create two Labelbox  projects
# Project defaults to batch mode with benchmark quality settings if queue mode argument is not provided
# Note that queue mode will be deprecated once dataset mode is deprecated 

batch_project = client.create_project(name="batch-test-project",
                                description="a description",
                                media_type=MediaType.Image,
                                # Setup quality settings (Benchmarks)
                                auto_audit_number_of_labels=1,
                                queue_mode=QueueMode.Batch)

dataset_project = client.create_project(name="dataset-test-project",
                                description="a description",
                                media_type=MediaType.Image,
                                ## Setup quality settings (Consensus)
                                auto_audit_percentage=0.1,
                                auto_audit_number_of_labels=3,
                                queue_mode=QueueMode.Dataset)

dataset = client.create_dataset(name="queue_dataset")

# Add data to your dataset

In [21]:
## Example image
uploads = []

# Generate data rows
for i in range(1, 9):
  row = {
      "row_data":  f"https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_000{i}.jpeg",
      "global_key": f"ID-{uuid.uuid1()}"
    }
  uploads.append(row)


data_rows = dataset.create_data_rows(uploads)

data_rows.wait_till_done()
print("ERRORS dataset: " , data_rows.errors)

print("RESULT URL: ", data_rows.result_url)

ERRORS dataset2:  None
RESULT URL2:  https://storage.labelbox.com/cl3ahv73w1891087qbwzs3edd%2Fdata-row-imports-results%2Fcl9rhym4l3p4x07xy1gsz301t_cl9ria9z33t2q0702ftd39bdj.json?Expires=1666988187661&KeyName=labelbox-assets-key-3&Signature=e_DUH8TN5UuoCgtoNULdttXXM_k


# Attach data to your project and set data row priority

In [23]:
####### Connect your dataset 
# Dataset projects do not support batches
# LPO Will be deprecated for datasets, please use batches to use data row priority
dataset_project.datasets.connect(dataset)

######## Create batches 

# Batch projects do not support datasets

# Get all the data row ids from your dataset to create the batch
batch_datarows = [dr.uid for dr in list(dataset.export_data_rows())]


# Create the batch 

batch = batch_project.create_batch(
  "batch-demo", # Each batch in a project must have a unique name
  batch_datarows[0:2], # A list of data rows or data row ids
  5 # priority between 1(Highest) - 5(lowest) 5 is the max priority that can be set
)

batch2 = batch_project.create_batch(
  "batch-demo-2", # Each batch in a project must have a unique name
  #Provide a slice of the data since you can't import assets with global keys that already exist in the project.
  batch_datarows[2:4],
  1 # priority between 1(Highest) - 5(lowest) 5 is the max priority that can be set
)

batch3 = batch_project.create_batch(
  "batch-demo-3", # Each batch in a project must have a unique name
  # Provide a slice of the data since you can't import assets with global keys that already exist in the project.
  batch_datarows[4: len(batch_datarows)],
  2 # priority between 1(Highest) - 5(lowest) 5 is the max priority that can be set
)

print("Batch: ", batch)
print("Batch2: ", batch2)
print("Batch3: ", batch3)

Batch:  <Batch {'created_at': datetime.datetime(2022, 10, 27, 20, 31, 48, tzinfo=datetime.timezone.utc), 'name': 'batch-demo', 'size': 2, 'uid': '61ebcee0-5636-11ed-b875-39298baad172', 'updated_at': datetime.datetime(2022, 10, 27, 20, 31, 48, tzinfo=datetime.timezone.utc)}>
Batch2:  <Batch {'created_at': datetime.datetime(2022, 10, 27, 20, 31, 50, tzinfo=datetime.timezone.utc), 'name': 'batch-demo-2', 'size': 2, 'uid': '62de6c40-5636-11ed-8b12-7f90533b6a47', 'updated_at': datetime.datetime(2022, 10, 27, 20, 31, 50, tzinfo=datetime.timezone.utc)}>
Batch3:  <Batch {'created_at': datetime.datetime(2022, 10, 27, 20, 31, 52, tzinfo=datetime.timezone.utc), 'name': 'batch-demo-3', 'size': 4, 'uid': '64089460-5636-11ed-bcbd-1b9bfc00b2b5', 'updated_at': datetime.datetime(2022, 10, 27, 20, 31, 52, tzinfo=datetime.timezone.utc)}>


* <b>Go to your project and you should see the numbers appear in order.</b>

In [24]:
print("View the results here:",
      f"https://app.labelbox.com/projects/{batch_project.uid}")
print("View the results here:",
      f"https://app.labelbox.com/projects/{dataset_project.uid}")
# Click `start labeling` to see the images in order

View the results here: https://app.labelbox.com/projects/cl9rhx7a6339l07vd7rc366n6
View the results here: https://app.labelbox.com/projects/cl9rhx7xx3pso07xra8caeck2


## Remove Queue Order
- Remove all the batch priority in your project

In [27]:
batch_project.unset_labeling_parameter_overrides(dataset.export_data_rows())

True

# Cleanup

In [ ]:
# batch_project.delete()
# dataset_project.delete()
# dataset.delete()